# What's in an Avocado Toast: A Supply Chain Analysis

![](avocado_wallpaper.jpeg)

You find yourself in London, crafting a delectable avocado toast, a dish that has risen dramatically in popularity on breakfast menus since the 2010s. This straightforward recipe requires just five ingredients: a ripe avocado, half a lemon, a generous pinch of salt flakes, two slices of sourdough bread, and a good drizzle of extra virgin olive oil. Most of these ingredients are now staples in grocery stores, and as you will find with this project, that is no small feat!

In this project, you'll conduct a supply chain analysis of three ingredients used in avocado toast using the Open Food Facts database. This database contains extensive, openly-sourced information on various foods, including their origins. Through this analysis, you will gain an in-depth understanding of the complex supply chain involved in producing a single dish.

Three pairs of files are provided in the data folder:
- A CSV file for each ingredient, such as `avocado.csv`, with data about each food item and countries of origin.
- A TXT file for each ingredient, such as `relevant_avocado_categories`, containing only the category tags of interest for that food.

Here are some other key points about these files:
- Some of the rows of data in each of the three CSV files do not contain relevant data for your investigation. In each dataset, you will need to filter out rows with irrelevant data, based on values in the `categories_tags` column. Examples of categories are fruits, vegetables, and fruit-based oils. Filter the DataFrame to include only rows where `categories_tags` contains one of the tags in the relevant categories for that ingredient.
- Each row of data usually has multiple category tags in the `categories_tags` column.
There is a column in each CSV file called `origins_tags`, which contains strings for the country of origin of each item.

After completing this project, you'll be armed with a list of ingredients and their countries of origin and be well-positioned to launch into other analyses that explore how long, on average, these ingredients spend at sea.

[Open Food Facts database](https://world.openfoodfacts.org/)

In [2]:
import pandas as pd
import numpy as np 

In [3]:
# read in datasets needed

file_path = "/Users/Uche1/Desktop/Data/GitHub/supply_chain_analysis/data"

file_name = ["avocado.csv", "olive_oil.csv", "sourdough.csv", "relevant_olive_oil_categories.txt", "relevant_sourdough_categories.txt",
             "relevant_avocado_categories.txt"]

columns_to_read = ['code', 'lc', 'product_name_en', 'quantity', 'serving_size', 'packaging_tags', 'brands', 'brands_tags', 'categories_tags', 
                   'labels_tags', 'countries', 'countries_tags', 'origins', 'origins_tags']

avocado = pd.read_csv(f"{file_path}/{file_name[0]}", delimiter='\t', usecols= columns_to_read)

olive_oil = pd.read_csv(f"{file_path}/{file_name[1]}", delimiter='\t', usecols= columns_to_read)

sourdough = pd.read_csv(f"{file_path}/{file_name[2]}", delimiter='\t', usecols= columns_to_read)

sourdough_categories = pd.read_csv(f"{file_path}/{file_name[4]}", delimiter='\t', header= None)

olive_oil_categories = pd.read_csv(f"{file_path}/{file_name[3]}", delimiter='\t', header= None)

avocado_categories = pd.read_csv(f"{file_path}/{file_name[5]}", delimiter='\t', header= None)

/var/folders/fg/7ll5hhdd1817fzv12j0r4bfh0000gp/T/ipykernel_11657/1615206906.py:13: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  olive_oil = pd.read_csv(f"{file_path}/{file_name[1]}", delimiter='\t', usecols= columns_to_read)


In [4]:
avocado_categories = avocado_categories.rename(columns = {0: "categories_tags"}).reset_index(drop=True)
sourdough_categories = sourdough_categories.rename(columns={0:"categories_tags"}).reset_index(drop=True)
olive_oil_categories = olive_oil_categories.rename(columns = {0: "categories_tags"}).reset_index(drop=True)

In [55]:
type(avocado_categories)

pandas.core.frame.DataFrame

In [5]:
#drop all na rows 
avocado2 = avocado.loc[(avocado["origins_tags"].notna()) & (avocado["categories_tags"].notna()) & (avocado["countries"] == "United Kingdom")]

# turn columns having multiple categories and origin into a list of categories and origin
avocado3 = avocado2.copy()
avocado3["categories_tags"] = avocado2["categories_tags"].str.split(",")
avocado3["origins"] = avocado2["origins"].str.split(",")

avocado3.shape

(5, 14)

In [6]:
#create an empty list and add categoried by looping through the avocadoes_categories data frame 
import time 

start_time = time.time()
relevant_categories = []

for i, x in avocado_categories.iterrows():
    relevant_categories.append(x["categories_tags"])
print(relevant_categories)

end_time = time.time()

duration = start_time - end_time
print(f"the code took {duration} seconds")

['en:avocadoes', 'en:avocados', 'en:fresh-foods', 'en:fresh-vegetables', 'en:fruchte', 'en:fruits', 'en:raw-green-avocados', 'en:tropical-fruits', 'en:tropische-fruchte', 'en:vegetables-based-foods', 'fr:hass-avocados']
the code took -0.0015239715576171875 seconds


In [7]:
# we want to filter avocado data frame based on the list we have
# if avocado3 csv not in relevant_categories drop

matching_indices = []
for i, series in avocado3.iterrows():
    for x in series["categories_tags"]:
        if x in relevant_categories:
            matching_indices.append(i)
            break

avocado4 = avocado3.loc[matching_indices]           
avocado4

,code,lc,product_name_en,quantity,serving_size,packaging_tags,brands,brands_tags,categories_tags,labels_tags,countries,countries_tags,origins,origins_tags
361,00985833,en,Avacado,650 g,NaN,NaN,Marks & Spencer,marks-spencer,"[en:plant-based-foods-and-beverages, en:plant-...",NaN,United Kingdom,en:united-kingdom,[Peru],en:peru
508,5057172125395,en,Just Essentials Avocados,4pack,NaN,en:mixed-plastic-film-packet-to-recycle,Asda,asda,"[en:plant-based-foods-and-beverages, en:plant-...","en:class-i,en:contains-stones",United Kingdom,en:united-kingdom,[Peru],en:peru
510,23066755,en,Ready to Eat Avocados,2,NaN,en:mixed-plastic-bag,Asda,asda,"[en:plant-based-foods-and-beverages, en:plant-...",NaN,United Kingdom,en:united-kingdom,"[Spain, Peru]","en:spain,en:peru"
708,03201985,en,Avocado,2,80g,"en:mixed-plastic-packet,en:plastic-film",Tesco,tesco,"[en:plant-based-foods-and-beverages, en:plant-...","en:tesco-nurture,en:vitamin-e-source",United Kingdom,en:united-kingdom,"[Chile, Peru]","en:chile,en:peru"
1190,01600322,en,Ripe & ready medium avocados,2,NaN,"en:card-tray,en:ldpe-bag",Sainsbury's,sainsbury-s,"[en:plant-based-foods-and-beverages, en:plant-...",en:class-i,United Kingdom,en:united-kingdom,[Israel],en:israel


In [10]:
origins_dict = {}
for i, series in avocado4.iterrows():
    for x in series["origins"]:
        country = x.strip()
        if country not in origins_dict:
            origins_dict[country] = 1
        else:
            origins_dict[country] += 1
        
top_avocado_origin = max(origins_dict, key = origins_dict.get)
top_avocado_origin

'Peru'

## Testing area

In [11]:
avocado4["origins_tags"].value_counts()

en:peru             2
en:spain,en:peru    1
en:chile,en:peru    1
en:israel           1
Name: origins_tags, dtype: int64

In [74]:
# # testing a faster method using vectorization
# start_time = time.time()
# relevant_categories2 = avocado_categories["categories_tags"].to_list()

# print(relevant_categories2)

# end_time = time.time()

# duration = start_time - end_time
# print(f"the code took {duration} seconds")

['en:avocadoes', 'en:avocados', 'en:fresh-foods', 'en:fresh-vegetables', 'en:fruchte', 'en:fruits', 'en:raw-green-avocados', 'en:tropical-fruits', 'en:tropische-fruchte', 'en:vegetables-based-foods', 'fr:hass-avocados']
the code took -0.0006470680236816406 seconds


In [15]:
avocado4["origins"].to_list()

[['Peru'], ['Peru'], ['Spain', ' Peru'], ['Chile', ' Peru'], ['Israel']]

In [75]:
avocado["origins_tags"].notna()

0       False
1       False
2       False
3       False
4       False
        ...  
1780    False
1781    False
1782    False
1783    False
1784    False
Name: origins_tags, Length: 1785, dtype: bool

AttributeError: 'list' object has no attribute 'any'